In [1]:
import librosa
import datetime
import matplotlib.pyplot as plt
import numpy as np

import IPython.display as ipd
import librosa.display
import numpy as np
import math

import tarfile
#import zipfile

In [3]:
from data_processing.mozilla_common_voice import MozillaCommonVoiceDataset
from data_processing.dataset import Dataset
from data_processing.feature_extractor import FeatureExtractor
import warnings

In [191]:
##### my dataset #####

### downloading from mozilla website requires a fresh regenerated link
#!wget 'https://mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com/cv-corpus-6.1-2020-12-11/ru.tar.gz?...' -O ru.tar --no-check-certificate

--2021-02-01 00:32:01-- OK
Length: 3655676916 (3.4G) [application/x-tar]
Saving to: ‘ru.tar’

100%[====================================>] 3,655,676,916 9.73MB/s   in 6m 25s 

2021-02-01 00:38:27 (9.07 MB/s) - ‘ru.tar’ saved [3655676916/3655676916]



In [ ]:
tar = tarfile.open('../ru.tar', 'r|gz')
tar.extractall(path="../")
tar.close()

### ------------create dataset---------------

In [16]:
warnings.filterwarnings(action='ignore')

mozilla_basepath = './cv-corpus-6.1-2020-12-11/ru'

mcv = MozillaCommonVoiceDataset(mozilla_basepath, val_dataset_size=2000)
clean_train_filenames, clean_val_filenames = mcv.get_train_val_filenames()

Total number of training examples: 15481
# of Training clean files: 13481
# of  Validation clean files: 2000


In [17]:
windowLength = 256
config = {'windowLength': windowLength,
          'overlap': round(0.25 * windowLength),
          'fs': 16000,
          'audio_max_duration': 0.8}
## Create Test Set
clean_test_filenames = mcv.get_test_filenames()

Total number of training examples: 8007
# of Testing clean files: 8007


In [18]:
def mp3_wav(lst):
    ls = []
    for el in lst:
        ls.append(el[:-3] + 'wav')
    return ls

In [19]:
clean_test_filenames = mp3_wav(clean_test_filenames)
clean_train_filenames = mp3_wav(clean_train_filenames)
clean_val_filenames = mp3_wav(clean_val_filenames)

In [20]:
len(clean_train_filenames), len(clean_val_filenames), len(clean_test_filenames)

(13481, 2000, 8007)

In [12]:
clean_test_filenames[0]

'./cv-corpus-6.1-2020-12-11/ru/clips/common_voice_ru_19800825.wav'

In [14]:
test_dataset = Dataset(clean_test_filenames[:1001], **config)

In [15]:
test_dataset.create_tf_record(prefix='test', subset_size=1000, parallel=True)

Processing files from: 0 to 1000
Processing files from: 1000 to 2000


In [21]:
val_dataset = Dataset(clean_val_filenames, **config)
val_dataset.create_tf_record(prefix='val', subset_size=2000)

Processing files from: 0 to 2000


In [ ]:
train_dataset = Dataset(clean_train_filenames[:4001], **config)
train_dataset.create_tf_record(prefix='train', subset_size=4000)

Processing files from: 0 to 4000


In [ ]:
### removing leftovers
!rm ./records/test_1.tfrecords ./records/train_1.tfrecords